<a href="https://colab.research.google.com/github/adautofbn/ri_labs/blob/master/lab04/index_consulta.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install memory_profiler

import pandas as pd
import nltk
import re
import collections
import heapq
import time
from nltk.tokenize import RegexpTokenizer

%load_ext memory_profiler

nltk.download('stopwords')
result = pd.read_csv('https://raw.githubusercontent.com/adautofbn/ri_lab_01/master/output/results.csv') # Resultados adquiridos do site brasil-247

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### 1. Execute o algoritmo ilustrado na Fig. 5.8 do livro texto (pag. 157) para gerar um índice similar o mostrado na Fig. 5.4 (pag. 134). Guarde o índice em disco em formato csv.

In [0]:
tknz = RegexpTokenizer(r'([A-Za-zÁáÉéÍíÓóÚúÃãÕõÇçÂâÊê]{3,27})')
stopwords = nltk.corpus.stopwords.words('portuguese') 
indexes = {}
n = 0

for text in result.text:
  words = [word for word in tknz.tokenize(text.lower())
           if not bool(re.search(r'\d', word))
           and word not in stopwords and len(word) >= 3]  
  n += 1
  for t in words:
    if t not in indexes.keys():
      indexes[t] = []
    indexes[t].append(n)
    
for elem in indexes.items():
  d = dict(collections.Counter(elem[1]))
  indexes[elem[0]] = list(d.items())

Tokenização feita a partir de regex (aceitando palavras em português) e filtragem de palavras sem significado expressivo contidas na lista de stopwords.

Utilizando dicionário foi possível realizar a indexação dos tokens adquiridos, a cada novo documento tokenizado:
 * Númeração do documento a ser processado;
 * Remoção de palavras repetidas no documento fazendo o cast do array de tokens para um set;
 * As palavras que aparecem pela primeira são adicionadas como chave;
 * As palavras que já estão no dicionário tem o número do documento acrescentado nos valores da sua chave.

In [0]:
tokens_table = pd.DataFrame()

tokens_table['Word'] = [word for word in indexes.keys()]
tokens_table['Documents'] = [docs for docs in indexes.values()]

tokens_table.to_csv('ranking.csv', encoding='utf-8', index=False)

### 2. Implemente as abordagens de processamento de consulta documento-por-vez e termo-por-vez (Fig. 5.16 e 5.18).

In [0]:
def doc_a_time(query, indexes, k):
    start = time.time()
    
    query_indexes = [] # Array
    rank = [] # HeapQueue, implementation of PriorityQueue
    
    for word in query.split(" "):
      if word in indexes.keys():
          query_indexes.append(indexes[word])
    
    for doc in range(1, len(result.text)+1):
      doc_score = 0
      for index in query_indexes:
          for i in index:
              if i[0] == doc:
                  doc_score += i[1]
                  break
      if doc_score != 0:
        heapq.heappush(rank, (doc_score, doc))
    
    end = time.time() - start
    return heapq.nlargest(k, rank), end

In [0]:
def term_a_time(query, indexes, k):
    start = time.time()
    
    a = {} # Accumulator HashTable
    query_indexes = [] # Array
    rank = [] # HeapQueue, implementation of PriorityQueue
    
    for word in query.split(" "):
      if word in indexes.keys():
          query_indexes.append(indexes[word])
    
    for index in query_indexes:
      for i in index:
          doc = i[0]
          if doc not in a.keys():
              a[doc] = 0
          a[doc] += i[1]
    
    for (doc, score) in a.items():
      heapq.heappush(rank, (score, doc))
        
    end = time.time() - start
    return heapq.nlargest(k, rank), end

####Defina 5 consultas de um termo somente.


In [0]:
queries = ["bolsonaro", "educação", "ministério", "brasil", "economia"]

####Execute as 5 consultas em cada algoritmo retornando os top-10 documentos (parâmetro k do algoritmo).


In [0]:
k = 10
results_doc = []
results_term = []
times_doc = []
times_term = []

for query in queries:
  score_doc, time_doc = doc_a_time(query, indexes, k)
  results_doc.append(score_doc)
  times_doc.append(time_doc)
  
  score_term, time_term = term_a_time(query, indexes, k)
  results_term.append(score_term)
  times_term.append(time_term)

In [8]:
queries_df = pd.DataFrame()
queries_df['Query'] = queries
queries_df['Document at a time'] = results_doc
queries_df['Term at a time'] = results_term
queries_df['Comparative'] = queries_df['Document at a time'] == queries_df['Term at a time']
queries_df.index+=1
queries_df

,Query,Document at a time,Term at a time,Comparative
1,bolsonaro,"[(10, 62), (6, 49), (5, 70), (5, 63), (5, 28),...","[(10, 62), (6, 49), (5, 70), (5, 63), (5, 28),...",True
2,educação,"[(2, 56), (2, 12), (1, 58), (1, 32), (1, 15)]","[(2, 56), (2, 12), (1, 58), (1, 32), (1, 15)]",True
3,ministério,"[(6, 60), (3, 12), (2, 61), (2, 31), (2, 30), ...","[(6, 60), (3, 12), (2, 61), (2, 31), (2, 30), ...",True
4,brasil,"[(11, 59), (7, 66), (7, 33), (7, 21), (7, 7), ...","[(11, 59), (7, 66), (7, 33), (7, 21), (7, 7), ...",True
5,economia,"[(6, 6), (3, 32), (2, 29), (2, 8), (2, 3), (1,...","[(6, 6), (3, 32), (2, 29), (2, 8), (2, 3), (1,...",True


####Compare os tempos médios de execução e uso de memória de cada algoritmo.


In [9]:
queries_tempo_df = pd.DataFrame()
queries_tempo_df['Average time: Document at a time (ms)'] = times_doc
queries_tempo_df['Average time: Term at a time (ms)'] = times_term
queries_tempo_df.index+=1
queries_tempo_df

,Average time: Document at a time (ms),Average time: Term at a time (ms)
1,0.000302,0.000037
2,0.000096,0.000009
3,0.000213,0.000021
4,0.000221,0.000032
5,0.000147,0.000020


In [10]:
print('Documento por vez uso de memória:')
%memit doc_a_time(queries[0], indexes, k)

print('Termo por vez uso de memória:')
%memit term_a_time(queries[0], indexes, k)

Documento por vez uso de memória:
peak memory: 179.69 MiB, increment: 0.18 MiB
Termo por vez uso de memória:
peak memory: 179.69 MiB, increment: 0.00 MiB


### 3. Implemente uma das versões de consulta conjuntiva (AND) (Fig. 5.20 ou 5.21).

In [0]:
def conj_query(query, indexes, k):
  
  query_indexes = [] # Array
  rank = [] # HeapQueue, implementation of PriorityQueue
  
  for word in query.split(" "):
    if word in indexes.keys():
        query_indexes.append(indexes[word])
  
  all_indexes = [item for sublist in query_indexes for item in sublist]
  all_indexes.sort() # Sorted Array of all indexes from query terms
  
  for i in range(len(all_indexes)):
    doc_score = 0
    d = all_indexes.pop()
    repeat = 1
    for index in all_indexes:
      if index[0] == d[0]:
        doc_score += index[1]
        repeat += 1
    if doc_score != 0 and repeat == len(query_indexes):
      doc_score += d[1]
      heapq.heappush(rank, (doc_score, d[0]))
        
  return heapq.nlargest(k, rank)

####Defina 5 consultas conjuntivas (AND).


In [0]:
queries_conj = ["decisão sobre", "ministro paulo", "brasil país", "jair bolsonaro", "presidente disse"]

####Execute as 5 consultas em cada algoritmo retornando os top-10 documentos (parâmetro k do algoritmo).


In [0]:
k = 10
results_conj = []

for query in queries_conj:
  score_conj = conj_query(query, indexes, k)
  results_conj.append(score_conj)

In [14]:
conj_queries_df = pd.DataFrame()
conj_queries_df['Query'] = queries_conj
conj_queries_df['Results'] = results_conj
conj_queries_df.index+=1
conj_queries_df

,Query,Results
1,decisão sobre,"[(6, 65), (6, 55), (5, 8), (5, 5), (4, 62), (4..."
2,ministro paulo,"[(6, 56), (6, 8), (5, 75), (5, 68), (5, 14), (..."
3,brasil país,"[(14, 59), (11, 33), (9, 21), (9, 7), (8, 66),..."
4,jair bolsonaro,"[(12, 62), (8, 49), (7, 70), (6, 66), (6, 63),..."
5,presidente disse,"[(9, 55), (9, 12), (8, 63), (8, 59), (7, 70), ..."


####Dê evidências de que sua implementação está correta.

  No caso abaixo, podemos olhar separadamente para a lista invertida de cada termo e ver manualmente que todos os termos aparecem apenas no documento 1. O algoritmo realiza a soma dos scores de cada termo e atribui como score do documento no resultado final.

In [15]:
query = "petrobrás quer alterar estatuto"
query_terms = query.split()
terms_indexes = [indexes[term] for term in query_terms]
  
score_conj = conj_query(query,indexes,10)
score_conj = (score_conj[0][::-1])

query_terms.append('')
terms_indexes.append('')
a = ['', '', '', '', query]
b = ['', '', '', '', score_conj]

query_df = pd.DataFrame()
query_df['Terms'] = query_terms
query_df['Inverted List'] = terms_indexes
query_df['Query'] = a
query_df['Result'] = b
query_df.index += 1
query_df


,Terms,Inverted List,Query,Result
1,petrobrás,"[(1, 4), (48, 1), (61, 3)]",,
2,quer,"[(1, 1), (18, 2), (37, 3), (45, 2), (59, 1)]",,
3,alterar,"[(1, 1), (63, 1)]",,
4,estatuto,"[(1, 3)]",,
5,,,petrobrás quer alterar estatuto,"(1, 9)"
